In [5]:
import polars as pl

def load_and_aggregate(path_to_blocks, path_to_bundles):
    # Load the parquet files into Polars DataFrames
    blocks_df = pl.read_parquet(path_to_blocks)
    bundles_df = pl.read_parquet(path_to_bundles)

    # Aggregate PNL by unique builder using 'builder_address' and 'builder_profit_usd'
    pnl_by_builder = blocks_df.groupby("builder_address").agg([
        pl.col("builder_profit_usd").sum().alias("total_builder_profit_usd")
    ])

    # Aggregate PNL by unique searcher using 'eoa' and 'profit_usd'
    pnl_by_searcher = bundles_df.groupby("mev_contract").agg([
        pl.col("profit_usd").sum().alias("total_searcher_profit_usd")
    ])

    return pnl_by_builder, pnl_by_searcher

if __name__ == "__main__":
    path_to_blocks = "../db/parquet/block_table.parquet"
    path_to_bundles = "../db/parquet/bundle_table.parquet"

    pnl_by_builder, pnl_by_searcher = load_and_aggregate(path_to_blocks, path_to_bundles)

    # Optionally, print the results
    print("PNL by Builder:")
    print(pnl_by_builder)
    print("\nPNL by Searcher:")
    print(pnl_by_searcher)


PNL by Builder:
shape: (1, 2)
┌───────────────────────────────────┬──────────────────────────┐
│ builder_address                   ┆ total_builder_profit_usd │
│ ---                               ┆ ---                      │
│ str                               ┆ f64                      │
╞═══════════════════════════════════╪══════════════════════════╡
│ 0xDAFEA492D9c6733ae3d56b7Ed1ADB6… ┆ -4.640045                │
└───────────────────────────────────┴──────────────────────────┘

PNL by Searcher:
shape: (3, 2)
┌───────────────────────────────────┬───────────────────────────┐
│ mev_contract                      ┆ total_searcher_profit_usd │
│ ---                               ┆ ---                       │
│ str                               ┆ f64                       │
╞═══════════════════════════════════╪═══════════════════════════╡
│ 0x2Ec705D306b51e486B1bC0D6ebEE70… ┆ 964.068259                │
│ 0x4E69A51f24F5A46919113cc78Ab262… ┆ 365.797458                │
│ null               

In [7]:
def load_and_aggregate(path_to_blocks, path_to_bundles):
    # Load the parquet files into Polars DataFrames
    blocks_df = pl.read_parquet(path_to_blocks)
    bundles_df = pl.read_parquet(path_to_bundles)

    # Aggregate PNL by unique builder
    pnl_by_builder = blocks_df.groupby("builder_address").agg([
        pl.col("builder_profit_usd").sum().alias("total_builder_profit_usd")
    ])

    # Aggregate PNL by unique searcher (mev_contract)
    pnl_by_searcher = bundles_df.groupby("mev_contract").agg([
        pl.col("profit_usd").sum().alias("total_searcher_profit_usd")
    ])

    # Total MEV Count by Block
    total_mev_count_by_block = blocks_df.groupby("block_number").agg([
        pl.col("mev_count").sum().alias("total_mev_count")
    ])

    # Detail for each searcher MEV contract per block (assuming 'block_number' is present in bundles_df)
    # Here, it's assumed bundles_df contains a 'block_number' column for grouping by block.
    # Adjust the column name if your schema uses a different identifier.
    searcher_details_by_block = bundles_df.groupby(["block_number", "mev_contract"]).agg([
        pl.col("profit_usd").sum().alias("total_profit_usd")
    ])

    return pnl_by_builder, pnl_by_searcher, total_mev_count_by_block, searcher_details_by_block

if __name__ == "__main__":
    path_to_blocks = "../db/parquet/block_table.parquet"
    path_to_bundles = "../db/parquet/bundle_table.parquet"

    pnl_by_builder, pnl_by_searcher, total_mev_count_by_block, searcher_details_by_block = load_and_aggregate(path_to_blocks, path_to_bundles)

    # Optionally, print the results
    print("PNL by Builder:")
    print(pnl_by_builder)
    print("\nPNL by Searcher:")
    print(pnl_by_searcher)
    print("\nTotal MEV Count by Block:")
    print(total_mev_count_by_block)
    print("\nSearcher Details by Block:")
    print(searcher_details_by_block)


PNL by Builder:
shape: (1, 2)
┌───────────────────────────────────┬──────────────────────────┐
│ builder_address                   ┆ total_builder_profit_usd │
│ ---                               ┆ ---                      │
│ str                               ┆ f64                      │
╞═══════════════════════════════════╪══════════════════════════╡
│ 0x4838B106FCe9647Bdf1E7877BF73cE… ┆ 30.615116                │
└───────────────────────────────────┴──────────────────────────┘

PNL by Searcher:
shape: (2, 2)
┌───────────────────────────────────┬───────────────────────────┐
│ mev_contract                      ┆ total_searcher_profit_usd │
│ ---                               ┆ ---                       │
│ str                               ┆ f64                       │
╞═══════════════════════════════════╪═══════════════════════════╡
│ 0xD249942f6d417CbfdcB792B1229353… ┆ -25.144508                │
│ null                              ┆ -10127.245109             │
└────────────────────

In [8]:
import polars as pl

def read_data(path_to_blocks, path_to_bundles):
    blocks_df = pl.read_parquet(path_to_blocks)
    bundles_df = pl.read_parquet(path_to_bundles)
    return blocks_df, bundles_df

def aggregate_pnl(blocks_df, bundles_df):
    pnl_by_builder = blocks_df.groupby("builder_address").agg([
        pl.col("builder_profit_usd").sum().alias("total_builder_profit_usd")
    ])
    
    pnl_by_searcher = bundles_df.groupby("mev_contract").agg([
        pl.col("profit_usd").sum().alias("total_searcher_profit_usd")
    ])
    
    return pnl_by_builder, pnl_by_searcher

def total_mev_count_by_block(blocks_df):
    return blocks_df.groupby("block_number").agg([
        pl.col("mev_count").sum().alias("total_mev_count")
    ])

def searcher_details_by_block(bundles_df):
    return bundles_df.groupby(["block_number", "mev_contract"]).agg([
        pl.col("profit_usd").sum().alias("total_profit_usd")
    ])

if __name__ == "__main__":
    path_to_blocks = "../db/parquet/block_table.parquet"
    path_to_bundles = "../db/parquet/bundle_table.parquet"

    blocks_df, bundles_df = read_data(path_to_blocks, path_to_bundles)
    pnl_by_builder, pnl_by_searcher = aggregate_pnl(blocks_df, bundles_df)
    total_mev_counts = total_mev_count_by_block(blocks_df)
    searcher_details = searcher_details_by_block(bundles_df)

    print("PNL by Builder:")
    print(pnl_by_builder)
    print("\nPNL by Searcher:")
    print(pnl_by_searcher)
    print("\nTotal MEV Count by Block:")
    print(total_mev_counts)
    print("\nSearcher Details by Block:")
    print(searcher_details)


PNL by Builder:
shape: (1, 2)
┌───────────────────────────────────┬──────────────────────────┐
│ builder_address                   ┆ total_builder_profit_usd │
│ ---                               ┆ ---                      │
│ str                               ┆ f64                      │
╞═══════════════════════════════════╪══════════════════════════╡
│ 0x4838B106FCe9647Bdf1E7877BF73cE… ┆ 30.615116                │
└───────────────────────────────────┴──────────────────────────┘

PNL by Searcher:
shape: (2, 2)
┌───────────────────────────────────┬───────────────────────────┐
│ mev_contract                      ┆ total_searcher_profit_usd │
│ ---                               ┆ ---                       │
│ str                               ┆ f64                       │
╞═══════════════════════════════════╪═══════════════════════════╡
│ null                              ┆ -10127.245109             │
│ 0xD249942f6d417CbfdcB792B1229353… ┆ -25.144508                │
└────────────────────